#Preparing a custom pipeline for data preparation and CNN training.

For the dataset:

Use the TinySOL dataset: https://zenodo.org/records/3685367.

You can download this by using the following commands:

```
 user:~$ wget https://zenodo.org/records/3685367/files/TinySOL.tar.gz
 user:~$ wget https://zenodo.org/records/3685367/files/TinySOL_metadata.csv
 user:~$ unzip TinySOL.tar.gz && rm TinySOL.tar.gz
```



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio
from torchaudio.transforms import MelSpectrogram
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 125 * 125, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [ ]:
import os
import pandas as pd
import torchaudio
from torchaudio.transforms import MelSpectrogram

class CustomDataset(Dataset):
    def __init__(self, csv_path, audio_folder):
        self.df = pd.read_csv(csv_path)
        self.audio_folder = audio_folder
        self.mel_transform = MelSpectrogram(sample_rate=22050, n_fft=400, hop_length=160, n_mels=128)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        file_name = self.df.iloc[index]['Path']
        audio_path = os.path.join(self.audio_folder, file_name)

        waveform, sample_rate = torchaudio.load(audio_path, normalize=True)

        mel_spectrogram = self.mel_transform(waveform)

        ground_truth = None
        pseudo_label = None

        return {
            'file': file_name,
            'audio': waveform,
            'mel': mel_spectrogram,
            'gt': ground_truth,
            'pseudo': pseudo_label
        }


In [ ]:
import pandas as pd

df = pd.read_csv('TinySOL_metadata.csv')
print(df.columns)


Index(['Path', 'Fold', 'Family', 'Instrument (abbr.)', 'Instrument (in full)',
       'Technique (abbr.)', 'Technique (in full)', 'Pitch', 'Pitch ID',
       'Dynamics', 'Dynamics ID', 'Instance ID', 'String ID (if applicable)',
       'Needed digital retuning'],
      dtype='object')


In [ ]:
from google.colab import files

uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
import zipfile
import io

zip_filename = 'TinySQL.zip'

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('TinySQL')


In [ ]:
train_dataset = CustomDataset(csv_path='TinySOL_metadata.csv', audio_folder='TinySOL/audio')
pseudo_dataset = CustomDataset(csv_path='TinySOL_metadata.csv', audio_folder='TinySOL/audio')

train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
pseudo_loader = DataLoader(pseudo_dataset, batch_size=32, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


RuntimeError: Failed to open the input "TinySOL/audio/Winds/Sax_Alto/ordinario/ASax-ord-D#4-pp-N-N.wav" (No such file or directory).

In [ ]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

NameError: name 'dataset' is not defined

In [ ]:
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        inputs, labels = batch['mel'], batch['gt']
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

torch.save(model.state_dict(), 'correct_model.pth')

model.load_state_dict(torch.load('correct_model.pth'))
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    for batch in pseudo_loader:
        inputs, pseudo_labels = batch['mel'], batch['pseudo']
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, pseudo_labels)
        loss.backward()
        optimizer.step()

torch.save(model.state_dict(), 'pseudo_model.pth')

In [ ]:
model.load_state_dict(torch.load('correct_model.pth'))
model.eval()
correct_labels, predictions = [], []
with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch['mel'], batch['gt']
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct_labels.extend(labels.numpy())
        predictions.extend(preds.numpy())

accuracy = accuracy_score(correct_labels, predictions)
conf_matrix = confusion_matrix(correct_labels, predictions)

print(f'Accuracy with correct labels: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')

In [ ]:
model.load_state_dict(torch.load('pseudo_model.pth'))
model.eval()
pseudo_labels, pseudo_predictions = [], []
with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch['mel'], batch['gt']
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        pseudo_labels.extend(labels.numpy())
        pseudo_predictions.extend(preds.numpy())

pseudo_accuracy = accuracy_score(pseudo_labels, pseudo_predictions)
pseudo_conf_matrix = confusion_matrix(pseudo_labels, pseudo_predictions)

print(f'Accuracy with pseudo labels: {pseudo_accuracy}')
print(f'Confusion Matrix:\n{pseudo_conf_matrix}')

NameError: name 'model' is not defined